In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob
!pip install tensorflow_hub 
import numpy as np
import cv2
import PIL.Image as Image
import os
import random
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#InceptionetV2 model tutorial


In [ ]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# get all the directories of the data
train_dir= '/content/drive/MyDrive/Colab Notebooks/primary_small/train'
valid_dir= '/content/drive/MyDrive/Colab Notebooks/primary_small/valid'
test_dir= '/content/drive/MyDrive/Colab Notebooks/primary_small/test'
classes = ["brightpixel", "narrowband","narrowbanddrd","noise","squarepulsednarrowband","squiggle","squigglesquarepulsednarrowband"]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1/255.0) #,
                                     #width_shift_range=0.15,
                                     #height_shift_range=0.15,
                                     #zoom_range=0.2,
                                     #fill_mode = 'nearest')  
train_dataset = train_gen.flow_from_directory(batch_size=64,
                                                 directory=train_dir,
                                                 shuffle=True,
                                                 color_mode = 'rgb',
                                                 target_size=(200, 200), 
                                                 class_mode='categorical')

validation_gen = ImageDataGenerator(rescale=1/255.0)
validation_dataset = validation_gen.flow_from_directory(batch_size=64,
                                                 directory=valid_dir,
                                                 shuffle=True,
                                                 color_mode = 'rgb',
                                                 target_size=(200, 200), 
                                                 class_mode='categorical')

Found 5600 images belonging to 7 classes.
Found 700 images belonging to 7 classes.


In [ ]:
import os
from keras import applications

In [ ]:
import tensorflow as tf
def create_inception_model():
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    base_model = InceptionV3(input_shape = (200, 200, 3), include_top = False, weights = 'imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    # for layer_idx in range(len(pretrained_model.layers)):
    #     if layer_idx not in [1,2,3,305,306,307,308,309,310]:
    #         pretrained_model.layers[layer_idx].trainable = False
    from tensorflow.keras.optimizers import RMSprop
    from tensorflow.keras import layers
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = layers.Dense(7, activation='softmax')(x)
    #model = tf.keras.models.Model(base_model.input, x)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)

    model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
inception_model=create_inception_model()
#inception_model.summary()
#tf.keras.utils.plot_model(inception_model, 'model.png', show_shapes= True)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
history = inception_model.fit_generator(
  train_dataset,
  steps_per_epoch = 64,
  epochs = 30,
  validation_data = validation_dataset,
  #validation_steps = 32,
  verbose = 1,
)

<ipython-input-9-f0a5dfef2376>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = inception_model.fit_generator(


Epoch 1/30
64/64 [==============================] - 929s 14s/step - loss: 1.6719 - accuracy: 0.4451 - val_loss: 1.2047 - val_accuracy: 0.5614
Epoch 2/30
64/64 [==============================] - 429s 7s/step - loss: 1.2261 - accuracy: 0.5430 - val_loss: 1.2109 - val_accuracy: 0.5743
Epoch 3/30
64/64 [==============================] - 419s 7s/step - loss: 1.1367 - accuracy: 0.5878 - val_loss: 1.1403 - val_accuracy: 0.5829
Epoch 4/30
64/64 [==============================] - 417s 7s/step - loss: 1.0346 - accuracy: 0.6252 - val_loss: 1.1737 - val_accuracy: 0.5729
Epoch 5/30
64/64 [==============================] - 416s 7s/step - loss: 0.9581 - accuracy: 0.6509 - val_loss: 1.1250 - val_accuracy: 0.5614
Epoch 6/30
64/64 [==============================] - 421s 7s/step - loss: 0.8904 - accuracy: 0.6682 - val_loss: 1.1460 - val_accuracy: 0.5843
Epoch 7/30
64/64 [==============================] - 415s 6s/step - loss: 0.8161 - accuracy: 0.7037 - val_loss: 1.1297 - val_accuracy: 0.5757
Epoch 8/30
6

In [ ]:
from tensorflow.python.ops.math_ops import truncate_div
test_datagen = ImageDataGenerator(rescale=1/255.0)
test_generator = test_datagen.flow_from_directory(batch_size=64,
                                                 directory=test_dir,
                                                 shuffle=True,
                                                 color_mode = 'rgb',
                                                 target_size=(200, 200), 
                                                 class_mode='categorical')
class_label_mapping = {v:k for k,v in test_generator.class_indices.items()}
test_data = [next(test_generator) for i in range(700)]
test_data_X = [data[0] for data in test_data] 
test_data_X = np.array(np.squeeze(test_data_X, axis=1))
test_data_y = np.array([fname.split('/')[0] for fname in test_generator.filenames])
class_labels = list(set(test_data_y))

test_data_X.shape, test_data_y.shape

In [ ]:
def evaluate_model_results(model, test_data, test_labels, 
                           class_label_mapping, class_labels):
    predictions = model.predict(test_data, verbose=1)
    prediction_labels = [class_label_mapping[idx] for idx in predictions.argmax(axis=1)]
    print(classification_report(y_true=test_labels, y_pred=prediction_labels))
    return pd.DataFrame(confusion_matrix(y_true=test_labels, y_pred=prediction_labels, labels=class_labels),
                        index=class_labels, columns=class_labels)

In [ ]:
evaluate_model_results(inception_model, test_data_X, test_data_y, class_label_mapping, class_labels)

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)

predict = inception_model.predict(test_generator,steps = nb_samples)
predictions = predict.argmax(axis = 1)
true_labels = test_generator.classes

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
 

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
import sklearn
import itertools
import seaborn as sns
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score, classification_report

import matplotlib.pyplot as plt
import matplotlib.image as mpimg 

cm = sklearn.metrics.confusion_matrix(true_labels, predictions)
target_names = ['brightpixel', 'narrowband','narrowbanddrd', 'noise',  'squarepulsednarrowband', 'squiggle','squigglesquarepulsednarrowband']
plot_confusion_matrix(cm,target_names)

In [ ]:
sns.heatmap(cm, annot = True, fmt = 'd')